In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
import matplotlib
from glob import glob
from timegan import timegan
# 2. Data loading
from data_loading import real_data_loading, sine_data_generation
# 3. Metrics
from metrics.discriminative_metrics import discriminative_score_metrics
from metrics.predictive_metrics import predictive_score_metrics
from metrics.visualization_metrics import visualization
from metrics.metric utils import *
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

In [3]:
nodelabels=['timestamp', 'Stempel_innen_mitte', 'Stempel_aussen', 'Matrize_zarge_oben', 'Matrize_zarge_mitte','Matrize_zarge_unten', 'Werkstueck_boden', 'Werkstueck_zarge_unten' , 'Werkstueck_zarge_mitte', 'Werkstueck_zarge_oben']

filenames = glob("../data/FEM_Data/*.csv")
print(len(filenames))
train_df = []
for filename in filenames:
    df = pd.read_csv(filename, names=nodelabels, skiprows=1, index_col=False).drop_duplicates()#.drop(columns='timestamp')
    train_df.append(df.to_numpy())

print(len(train_df))   
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_df) 
print(train_data.shape)

367


/home/fneubuerger/.local/anaconda3/envs/TGAN/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


367
(367, 305, 10)


In [4]:
## Newtork parameters
parameters = dict()

parameters['module'] = 'gru' 
parameters['hidden_dim'] = 24
parameters['num_layer'] = 3
parameters['iterations'] = 10000
parameters['batch_size'] = 128

In [ ]:
generated_data = timegan(train_data, parameters)   
print('Finish Synthetic Data Generation')





Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons


OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-11
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #216: KMP_AFFINITY: x2APIC ids not unique.
OMP: Info #216: KMP_AFFINITY: decoding legacy APIC ids.
OMP: Info #216: KMP_AFFINITY: legacy APIC ids not unique.
OMP: Info #220: KMP_AFFINITY: parsing /proc/cpuinfo.
OMP: Info #157: KMP_AFFINITY: 12 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #287: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #192: KMP_AFFINITY: 1 socket x 6 cores/socket x 2 threads/core (6 total cores)
OMP: Info #218: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 1 maps to socket 0 core 0 thread 1 
OMP: Info #172: KMP_AFFINITY: OS proc 2 maps to socket 0 core 1 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 3 maps to socket 0 core 1 thread 1 
OMP: Info #172: KMP_AFFINITY: OS p

Start Embedding Network Training


OMP: Info #254: KMP_AFFINITY: pid 408 tid 455 thread 1 bound to OS proc set 2
OMP: Info #254: KMP_AFFINITY: pid 408 tid 457 thread 2 bound to OS proc set 4
OMP: Info #254: KMP_AFFINITY: pid 408 tid 459 thread 4 bound to OS proc set 8
OMP: Info #254: KMP_AFFINITY: pid 408 tid 458 thread 3 bound to OS proc set 6
OMP: Info #254: KMP_AFFINITY: pid 408 tid 460 thread 5 bound to OS proc set 10
OMP: Info #254: KMP_AFFINITY: pid 408 tid 461 thread 6 bound to OS proc set 1
OMP: Info #254: KMP_AFFINITY: pid 408 tid 462 thread 7 bound to OS proc set 3
OMP: Info #254: KMP_AFFINITY: pid 408 tid 464 thread 9 bound to OS proc set 7
OMP: Info #254: KMP_AFFINITY: pid 408 tid 463 thread 8 bound to OS proc set 5
OMP: Info #254: KMP_AFFINITY: pid 408 tid 465 thread 10 bound to OS proc set 9
OMP: Info #254: KMP_AFFINITY: pid 408 tid 466 thread 11 bound to OS proc set 11
OMP: Info #254: KMP_AFFINITY: pid 408 tid 467 thread 12 bound to OS proc set 0
OMP: Info #254: KMP_AFFINITY: pid 408 tid 454 thread 13 bou

step: 0/10000, e_loss: 314.4693


In [ ]:
metric_iteration = 5

discriminative_score = list()
for _ in range(metric_iteration):
    temp_disc = discriminative_score_metrics(train_data, generated_data)
    discriminative_score.append(temp_disc)

print('Discriminative score: ' + str(np.round(np.mean(discriminative_score), 4)))

In [ ]:
predictive_score = list()
for tt in range(metric_iteration):
    temp_pred = predictive_score_metrics(train_data, generated_data)
    predictive_score.append(temp_pred)   
    
print('Predictive score: ' + str(np.round(np.mean(predictive_score), 4)))

In [ ]:
scores = pd.DataFrame({'DiscriminativeScore' : discriminative_score, 'PredictiveScore' : predictive_score})

In [ ]:
visualization(ori_data, generated_data, 'pca', save=False)
visualization(ori_data, generated_data, 'tsne', save=False)